<a href="https://colab.research.google.com/github/bobby-py2002/FoodVision/blob/main/scrape.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

scrape.py



In [1]:
import requests
from bs4 import BeautifulSoup
import json

def clean_text(text):

    return ' '.join(text.split()).replace('\n', ' ').strip()

def scrape_recipe(url):
  try:
    response = requests.get(url , timeout = 30)
    response.raise_for_status()
    print(response.raise_for_status())
    soup = BeautifulSoup(response.text, 'html.parser')

    name = (soup.find('h1', class_= 'recipe-title')
            or
            soup.select_one('h1[data-testid="recipe-title"]')
            or
            soup.find('h1')
            )
    if name:
      name = name.text.strip()

    ingredients = [clean_text(ing.text) for ing in soup.select('.ingredient-list li')]
    instructions = [clean_text(ins.text) for ins in soup.select('.direction-list li')]
    return {
        'name': name,
        'ingredients': ingredients,
        'instructions': instructions
    }
  except requests.exceptions.Timeout:
     print(f"🕒 Timeout: {url} (server too slow)")
     return None
  except requests.exceptions.HTTPError as e:
    print(f"🚨 HTTP {e.response.status_code} error: {url}")
    return None
  except Exception as e:
    print(f"💥 Unexpected error on {url}: {str(e)}")

recipes = []
urls = ['https://www.food.com/recipe/bagel-french-toast-casserole-362199']

for url in urls:
    try:
        recipes.append(scrape_recipe(url))
    except Exception as e:
        print(f"Error: {e}")

with open('/home/recipes.json', 'w', encoding='utf-8') as f:
    json.dump(recipes, f, indent=2, ensure_ascii=False)

None
